# Imports and configuration.


In [1]:
%matplotlib inline

In [2]:
import collections
import lda
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import psycopg2
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text

import common

import functions as f



In [3]:
matplotlib.style.use('ggplot')

# Load data

In [7]:
## TODO: fix unicode here.
df = f.load_data()



Loaded 16539 records in total


Loaded 4506 records from epicurious.com
Loaded 12033 records from allrecipes.com

## Examining and cleaning data

In [8]:
df = df[df['ingredient_txt'].str.len() > 20]
print df['ingredient_txt'].str.len().describe()
plt.figure()
df['ingredient_txt'].str.len().plot(kind='hist').set_title('Ingredients character count')
plt.savefig('character-counts.png')
df = df.reset_index()




count    16526.000000
mean       296.671366
std        127.103774
min         23.000000
25%        206.000000
50%        280.000000
75%        371.000000
max       1087.000000
Name: ingredient_txt, dtype: float64


In [9]:
## clean up quoting.
pattern = "[\"\']"
for k in ['title', 'ingredient_txt', 'url', 'image']:
    df[k] = df[k].str.replace(pattern, '')

## formatting ingredients.
df['ingredient_txt'] = df['ingredient_txt'].str.replace('\n',' ')


In [10]:
def rm_stopwords(stopwords, x):
    return ' '.join([w for w in x.split() if w.strip() not in stopwords])



In [11]:
## replace punctuation to improve tokenizing and stop word filtering.
df['ingredient_txt_no_stopwords'] = df['ingredient_txt'].str.replace('[\W]', ' ')
df['ingredient_txt_no_stopwords'] = map(lambda x: rm_stopwords(f.get_stop_words(), x), df['ingredient_txt_no_stopwords'])



In [12]:
# Extract features
# TODO: remove stopwords before vectorizing, because the stop words don't capture bigrams.

vectorizer = CountVectorizer(
stop_words='english'
, ngram_range=(1, 2)
, token_pattern='[A-Za-z]+'
, min_df = 10
, max_df = 0.25
)

features = vectorizer.fit_transform(df.ingredient_txt_no_stopwords)
## features is a document x term matrix.

wc = f.feature_counts(vectorizer, features)




In [13]:
wc.sort('count').tail(25).plot('word','count', kind='bar')
plt.savefig('word-counts.png')


In [14]:
m = lda.LDA(n_topics=40, random_state=0, n_iter=200)
m.fit(features)
print('Finished running model')


Finished running model


In [38]:
## select best number of topics using loglikelihood.
ll = {}
for k in range(5, 200, 5):
    print k
    mk  = lda.LDA(n_topics=k, random_state=0, n_iter=400)
    mk.fit(features)
    ll[k] = mk.loglikelihood()




95


90


85


80


75

...


30


25


20


15

5
10

In [95]:

ks = sorted(ll.keys())
vs = [ll[k] for k in ks]
plt.plot(ks, vs)


# Evaluating the model.
## Convergence

In [15]:
p = plt.figure()
plt.plot(m.loglikelihoods_, '-')
plt.title('Loglikelihood')
p.savefig('loglikelihood.png')


In [15]:
## assessing convergence.
n = features.shape[0]
size = int(n * 0.8)

i0 = np.random.random_integers(0, n, size)
i1 = np.random.random_integers(0, n, size)

f0 = features[i0, :]
f1 = features[i1, :]

print('running model on sample 0...')
m0 = lda.LDA(n_topics=40, random_state=0, n_iter=100)
m0.fit(f0)

print('running model on sample 1...')
m1 = lda.LDA(n_topics=40, random_state=0, n_iter=100)
m1.fit(f1)

print('Finished running models')


running model on sample 0...
running model on sample 1...


Finished running models

In [16]:
a0 = zip(i0, np.argmax(m0.doc_topic_, axis=1))
a1 = zip(i1, np.argmax(m0.doc_topic_, axis=1))

## make 

t0 = collections.defaultdict(set)
for doc, topic in a0:
    t0[topic].add(doc)

t1 = collections.defaultdict(set)
for doc, topic in a1:
    t1[topic].add(doc)


In [17]:
## filter out elements not in both sets.
canon = set(i0).union(set(i1))
for k,v in t0.iteritems():
    t0[k] = t0[k].intersection(canon)
for k,v in t1.iteritems():
    t1[k] = t1[k].intersection(canon)

## compare set assignments.


In [18]:
def diff(t0, t1):
    """Calculate set membership differences as a stability measure.
    """


    diff = 0
    for k0, x0 in t0.iteritems():
        # print 'topic:', k0
        d = [len(x0.difference(x1))/float(len(x0.union(x1))) for x1 in t1.values()]
        # print 'differences:', d
        # print 'min diff:', min(d)
        # print 'current diff', diff
        diff += min(d)
    return diff

In [19]:
print diff(t0, t1)

8.73957410405


# Assessing topics

In [16]:
## Extracting topic data.
## most probable words by topic.
## TODO: check if these are properly sorted within each topic.
w = f.most_probable_words(m, vectorizer.get_feature_names(), 10)
w.columns = ['rank','topic','word','prob']

## most probable documents by topic.
# np.apply_along_axis(lambda i: df.iloc[i]['title'], 1, doc_ids)
doc_ids = np.argsort(m.doc_topic_, axis=0)[-4:-1,:].T
doc_probs = np.sort(m.doc_topic_, axis=0)[-4:-1,:].T


In [986]:

f.show_topics(m, df, doc_probs, doc_ids, w)


topic: 39
documents:
 Slow Cooker German-Style Pork Roast with Sauer...      0.655
                        Molasses Brined Pork Chops    0.63125
               Herbed Pork Chops with Homemade Rub  0.6130435
1 3 parsley 1 4 marjoram 1 4 thyme 3 rubbed sage 2 2 onion 1 1 cinnamon 4 boneless pork loin chops 2 vegetable 1 canola-----1 2 kosher 1 2 molasses 4 1 boiling 7 cold 4 2 1 4 center bone pork chops 1 2 vegetable-----6 potatoes 1 1 3 boneless pork loin roast 1 32 jar sauerkraut liquid 2 caraway seeds
----------------------------------------------------------------------
 chops  pork chops  boneless  boneless pork   loin  roast  pork loin  vegetable  tenderloin
 0.069       0.053     0.052          0.046  0.035  0.033      0.031      0.022       0.022



topic: 35
documents:
            Slow Cooker Lasagna II  0.8818182
               Slow Cooker Lasagna  0.8741935
 Carol\u0027s Baked Ziti Casserole  0.8607143
1 16 package ziti pasta 1 egg 1 15 container ricotta cheese 1 4 Parmesan cheese 1 28 jar meatless spaghetti sauce 2 skim mozzarella cheese 1 4 Parmesan cheese-----1 lean beef 1 onion 2 1 29 tomato sauce 1 6 tomato paste 1 1 2 1 oregano 1 12 package lasagna noodles 12 cottage cheese 1 2 Parmesan cheese 16 mozzarella cheese-----1 16 package lasagna noodles 1 lean beef 1 1 2 26 jars spaghetti sauce 2 mozzarella cheese 1 2 Parmesan cheese 1 8 container ricotta cheese 2 eggs 2 mozzarella cheese
----------------------------------------------------------------------
 mozzarella cheese  parmesan  parmesan cheese  tomato  package  basil  spaghetti  pasta  oregano
             0.049     0.044            0.040   0.034    0.030  0.026      0.022  0.022    0.020
topic: 36
documents:
                   Cherry Tomato Vinaigrette  0.7833333
 Ch


topic: 31
documents:
 Southern Barbeque Pulled Beef Sandwiches  0.7594595
          Anyone Can Make BBQ Pulled Pork   0.730303
    Loose Meat on a Bun, Restaurant Style  0.6894737
3 beef 1 4 onion 3 Worcestershire sauce 4 beef broth 1 1 2 butter 12 hamburger buns split-----1 5 pork shoulder roast 5 2 strong brewed coffee 1 onion 2 Worcestershire sauce 2 liquid smoke flavoring 1 flakes 1 16 bottle barbeque sauce 12 sandwich buns split toasted-----cooking spray 1 2 beef broth 1 liquid smoke flavoring 1 1 4 boneless beef chuck roast 3 barbeque seasoning Grill Mates Smokehouse Maple 2 Worcestershire sauce 2 barbeque sauce 8 Kaiser rolls split
----------------------------------------------------------------------
 split  roast   buns  barbeque  barbeque sauce  hamburger  chuck  hamburger buns  sirloin
 0.033  0.031  0.027     0.024           0.023      0.023  0.021           0.021    0.019
topic: 32
documents:
             Spinach and Feta Turkey Burgers  0.7117647
                        


topic: 27
documents:
           Nutty Shamrock Shake  0.7958333
                 g\u0027Oatmeal  0.7909091
 Mediterranean Breakfast Quinoa  0.7833333
1 4 raw almonds 1 cinnamon 1 quinoa 2 milk 1 sea 1 vanilla extract 2 honey 2 pitted dates 5 apricots-----3 rolled oats 2 Irish steel oats 5 ripe bananas mashed 1 2 flax seeds 1 2 1 4 peanut butter 1 4 pitted dates optional 1 4 pecans optional 1 4 chia seeds 2 coconut 1 1 2 vanilla extract 1 2 1 3 honey-----1 1 2 pistachio ice cream 1 milk 1 2 almond extract 5 drops green food coloring Whipped cream optional 1 pistachios
----------------------------------------------------------------------
   egg  almonds  vanilla   milk  yolks  cinnamon  egg yolks  pitted    raw
 0.031    0.027    0.026  0.018  0.016     0.016      0.015   0.015  0.015
topic: 28
documents:
                Mom-Mom\u0027s Pepper Steak  0.6241379
 Stuffed Peppers with Turkey and Vegetables   0.596875
                            Cajun Chow Mein  0.5962963
1 lean beef 1 2 1 


topic: 25
documents:
 Lamb Shank Braised in White Wine with Rosemary      0.675
                               Forager Sandwich  0.6733333
                                 Parmesan Broth  0.6368421
1 head 1 onion saut onion 1 handful thyme Sprigs parsley 1 bay leaf 1 shake peppercorns 1 wine 1 Parmesan rinds 8-----1 2 shallot 1 sprig thyme 4 king trumpet mushrooms 2 1 4 scored 1 4 1 4 wine vinegar 1 1 2 kosher-----3 4 lamb shanks 5 1 onion 2 rosemary sprigs 1 1 wine
----------------------------------------------------------------------
 rosemary    bay  sprigs   wine   leaf  bay leaf  kosher  parsley  vinegar
    0.059  0.053   0.047  0.042  0.030     0.029   0.022    0.021    0.018
topic: 26
documents:
 Honey Glazed Stuffed Chicken  0.6529412
           Easy Chicken Curry  0.6529412
   A Good Easy Garlic Chicken    0.63125
3 butter 4 skinless boneless chicken breast halves 2 1 seasoning 1 onion-----4 skinless boneless chicken breast halves 4 Boursin cheese herbs 8 bacon toothpicks 1 

...


1 3 chicken 1 purpose flour 1 paprika 1 quart vegetable frying-----One 3 3 1 2 broiler fryer frying Note 2 1 4 3 4 unsifted purpose flour Vegetable frying lard Mrs McCollum used lard 1-----1 2 beef 1 purpose flour 4 eggs 3 vegetable needed 1 butter 1 purpose flour needed 1 milk
----------------------------------------------------------------------
 purpose  purpose flour   milk  vegetable  frying  butter purpose  needed    egg   eggs
   0.142          0.141  0.041      0.032   0.031           0.021   0.019  0.018  0.017
topic: 10
documents:
 Creamy Polenta with Arrabbiata Sausage Ragout  0.6609756
                  Rossi\u0027s Sausage Gnocchi  0.6483871
                     Spicy Breakfast Meatballs  0.6192308
1 20 package bulk spicy Italian turkey sausage 1 20 package spicy Italian turkey 3 eggs 4 2 fat Cheddar cheese 1 4 onion 1 4-----1 1 Italian sausage links 1 1 2 onion 1 1 1 2 Italian seasoning 1 14 tomatoes 2 1 4 1 sugar 1 Italian seasoning 1 1 package gnocchi flat leaf parsley


topic: 6
documents:
                Red Ryder  0.8555556
 Bill Hecks Old Fashioned       0.85
                  Mai Tai      0.844
1 silver gold rum 3 4 lime juice 1 2 aged rum 1 2 orange Cura ao premium triple sec 1 2 orgeat almond syrup 1 4 simple syrup Float dark Jamaican rum optional Mint sprig-----2 sugar 4 dashes Angostura bitters 1 lemon wheel 1 lime wheel 1 orange wheel 1 maraschino cherry 1 2 grenadine 1 4 bourbon rye whiskey Club soda Ginger ale-----1 750 milliliter bottle Campari 1 750 milliliter bottle dark rum 3 chilled brewed rooibos tea 3 pomegranate juice 3 unsweetened cranberry juice 1 1 2 Angostura bitters
----------------------------------------------------------------------
 orange   lime  orange juice  lime juice   zest  sugar  honey   mint  pineapple
  0.088  0.054         0.038       0.037  0.025  0.016  0.016  0.015      0.014
topic: 7
documents:
                   Portobello Bellybuttons      0.805
                  Clam Sauce with Linguine  0.7434783
 Escargo


----------------------------------------------------------------------
 shrimp  deveined  uncooked  shrimp deveined  grain   long  seasoning  cooked  long grain
  0.061     0.050     0.048            0.037  0.031  0.030      0.029   0.029       0.023
topic: 3
documents:
    Taco in a Bag  0.8741935
 Mexican Meatloaf       0.85
       Enchiladas      0.805
2 lean beef 1 onion 1 8 12 8 flour tortillas 2 vegetable 8 Colby cheese 2 19 cans enchilada sauce-----2 lean beef 1 1 25 package taco seasoning mix 1 16 refried beans 4 8 flour tortillas 3 4 salsa 1 2 Cheddar cheese-----1 lean beef 1 1 package taco seasoning mix 4 2 5 packages corn chips 2 lettuce 1 tomato 1 Cheddar cheese 1 3 salsa 1 2 sour cream
----------------------------------------------------------------------
 sour cream  cream  cheddar  cheddar cheese   corn  tortillas  package  flour    mix
      0.048  0.044    0.034           0.031  0.029      0.027    0.023  0.022  0.021
topic: 4
documents:
             Mexican Orange Ch


1 3 rabbit cleaned 1 free marinade Mrs Dash-----1 1 instant rice 1 butter optional 1 optional cooking spray-----1 egg 1 lean beef 1 quick cooking oats 1 onion 1 spinach 2 low fat Mexican blend cheese 4 grain buns
----------------------------------------------------------------------
 cooking  spray  cooking spray    fat  wheat    low  blend  low fat  vegetable
   0.101  0.077          0.071  0.030  0.027  0.024  0.021    0.021      0.019
topic: 1
documents:
   Carrots and Greens with Dilly Bean Vinaigrette      0.724
        Linguine and Clams with Almonds and Herbs      0.724
 Sliced Baguette with Radishes and Anchovy Butter  0.6714286
1 2 1 unsalted butter 2 3 anchovy fillets 2 chives Coarse kosher 16 1 2 diagonal baguette 10 radishes French Breakfast diagonal Additional chives-----1 4 dilly beans 1 pickling liquid 3 unsalted butter 1 1 1 2 carrots unpeeled 1 shallot 1 2 rings 3 bunches mustard greens 1 center ribs stems Kosher-----1 2 unsalted roasted almonds 2 chives 2 flat leaf p

topic: 0
documents:
   Popeye Power Burger      0.484
     Rice Quiche Crust  0.4733333
 Grilled Garlic Rabbit  0.4636364

In [971]:
# Plotting word distributions for each topic.
wb = f.most_probable_words(m, vectorizer.get_feature_names(), 10)
wb.columns = ['rank','topic','word','prob']

## make figure of word distributions for each topic.
g = sns.FacetGrid(wb, col='topic', col_wrap=10)
p = g.map(sns.barplot, 'word', 'prob')
## save figure for easier viewing.
p.savefig('word_dist_by_topic.png')

## TODO: figure out way of examining probs of words in relation to topic coherence:
## high average prob?

## make figure of document distributions for each topic.
## for each topic, show distribution of documents.





In [ ]:
## examine topics:
## 14: one very probable word.
## 32: many very probable words.
## 30: no very probable words.




In [18]:
## TODO: store one set of results for each run.

con = f.make_engine()

## massage document ids and probabilities into form suitable for database.
di = pd.DataFrame(doc_ids)
di['topic'] = di.index
di = pd.melt(di, id_vars='topic')
di.columns = ['topic','rank','recipe_key']

dp = pd.DataFrame(doc_probs)
dp['topic'] = dp.index
dp = pd.melt(dp, id_vars='topic')
dp.columns = ['topic','rank','prob']

dd = pd.merge(di, dp)
dd.to_sql('doc_prob', con, if_exists='replace')

# store recipes
df['key'] = df.index
## assign the most probable topic to each recipe.
df['topic'] = np.argmax(m.doc_topic_, axis=1)
df.to_sql('clean_recipes', con, if_exists='replace', index=False)

# store words
w.columns = ['rank','topic','word','prob']
w.to_sql('word_probs', con, if_exists='replace')


In [285]:

xx = pd.merge(df, dd, left_on='key', right_on='recipe_key', how='right')

## topics with low word probs.
## but, they seem pretty good.
print 'topics with low word probs.'
for n, g in xx[xx['topic'].isin([5,8,16,18,30])].groupby('topic'):
    print 'topic: %s' % n
    print g[['title','prob']].sort('prob').to_string()

##
print '='*80
print 'topics with one high word prob.'
print '='*80
for n, g in xx[xx['topic'].isin([1,4,9,14,21])].groupby('topic'):
    print 'topic: %s' % n
    print g[['title','prob']].sort('prob').to_string()




topics with low word probs.
topic: 5
                                          title      prob
31                       Fruit and Nut Pancakes  0.816667
27   Coconut Macaroon Sandwiches with Lime Curd  0.821212
116                        Joyce\u0027s Granola  0.823077
topic: 8
                                        title      prob
71      Spicy Coconut and Lime Grilled Shrimp  0.638636
98  Garlic, Cilantro, and Lime Sauteed Shrimp  0.669697
51                  Spicy Lime Grilled Shrimp  0.684000
topic: 16
                                                title      prob
25       Chocolate-Dipped Salted Caramel Marshmallows  0.765278
34  Maple-Gingerbread Layer Cake with Salted Maple...  0.811538
5                        Bourbon-Vanilla Marshmallows  0.816279
topic: 18
                                title      prob
78          Rajma (Kidney Bean Curry)  0.743299
43  Indian Chicken Curry (Murgh Kari)  0.753608
55           Indian Dahl with Spinach  0.762000
topic: 30
                    

In [469]:
## relationship between doc prob and length:
dpa = pd.DataFrame({'max_prob':np.max(m.doc_topic_, axis=1), 'topic':np.argmax(m.doc_topic_, axis=1)})
dpa = df.join(dpa)
dpa['ingredient_len'] = dpa['ingredient_txt'].str.len()

dpa.plot('ingredient_len', 'mdpa_prob', kind='scatter')

# g = sns.FacetGrid(dpa, col='topic', col_wrap=10)
# p = g.map(sns.barplot, 'word', 'prob')
# ## save figure for easier viewing.
# p.savefig('word_dist_by_topic.png')



In [39]:
## generating recipes.
reload(f)
ww = f.all_word_probs(m, vectorizer.get_feature_names())
ww.to_sql('all_word_probs', con, if_exists='replace')

In [40]:
for n, g in ww.groupby('label'):
    print n
    print [', '.join(np.random.choice(g['word'], size=5, p=g['prob'])) for _ in range(0,1)]
    

0
[u'vegetable chicken, chicken, broth, broth, broth']
1
[u'lemon peel, parsley, butter lemon, parsley lemon, halibut']
2
[u'base, needed, smoked, cover, giblets']
3
[u'soup mix, processed cheese, onion soup, onion cans, lean']
4
[u'vegetable, vanilla extract, egg yolks, cacao, rose']
5
[u'fluid bottle, steak, skirt steak, bottles, beer']
6
[u'jack, jack cheese, butter flour, fat free, cheddar monterey']
7
[u'cream package, chives, cheese sour, light, button']
8
[u'basil vegetable, thyme, oregano, balsamic vinegar, oregano']
9
[u'condensed cheddar, potato nuggets, tomato soup, mushroom, meat']
10
[u'molasses, vegetable, seeds, fennel, cooking spray']
11
[u'orzo, butter cans, hair, asiago cheese, flakes']
12
[u'seedless, duck, chicken, juiced, juiced']
13
[u'mushrooms, baby bella, heavy, cream, beef stock']
14
[u'purpose, purpose flour, milk, purpose, milk']
15
[u'worcestershire, cayenne, blue cheese, ketchup mustard, beef hot']
16
[u'hash brown, crust pie, pie, melted butter, butter me


[u'bell, bell, chile peppers, green, green']
